<a href="https://colab.research.google.com/github/ShekSingh/AlgorithmsPartOne/blob/master/Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt


N = 1000
np.random.seed(1)
id = np.random.choice(range(1, 11), N)
purchaseday = np.random.choice(range(1, 366), N)
purchasedate = []

months = [0,31,59,90,120,151,181,212,243,273,304,334]

def makedate(x):
    month = max([i for i in range(len(months)) if x > months[i]])
    if month > 0:
        day = x - months[month]
    else:
        day = x
    return pd.to_datetime("2022-"+str(month+1)+"-"+str(day))

for i in range(N):
    purchasedate.append(makedate(purchaseday[i]))

dat = pd.DataFrame({'id': id, 'purchasedate': purchasedate})
dat2 = dat.sort_values(['id', 'purchasedate'])
dat2['datediff'] = np.zeros(N)
dat2['salecat'] = np.empty(N, dtype='str')

for i in range(1, 11):
    dat2i = dat2[dat2['id']==i]
    datediff = np.diff(dat2i['purchasedate'])
    salecat = np.where(datediff <= pd.Timedelta(days=2), 'daily',
                       np.where(datediff <= pd.Timedelta(days=9), 'weekly',
                                np.where(datediff <= pd.Timedelta(days=17), 'biweekly', 'monthly')))
    dat2.loc[dat2['id']==i, 'datediff'] = np.concatenate(([0], pd.Series(datediff).dt.days))
    dat2.loc[dat2['id']==i, 'salecat'] = np.concatenate((['new'], salecat))

freqcounts = pd.DataFrame({'id': range(1, 11),
                           'daily': [np.sum(dat2.loc[dat2['id']==i, 'salecat']=='daily') for i in range(1, 11)],
                           'weekly': [np.sum(dat2.loc[dat2['id']==i, 'salecat']=='weekly') for i in range(1, 11)],
                           'biweekly': [np.sum(dat2.loc[dat2['id']==i, 'salecat']=='biweekly') for i in range(1, 11)],
                           'monthly': [np.sum(dat2.loc[dat2['id']==i, 'salecat']=='monthly') for i in range(1, 11)]})



alpha = np.ones(4)  # hyperparameter for the Dirichlet distribution
i = 0  # customer number

# Prior distribution: Dir(alpha1 = 1, alpha2 = 1,...)
# Posterior dist      Dir(alpha1 = 1+37, alpha2 = 1+52)

print(alpha.shape)
print(freqcounts.iloc[i,1:4].shape)



(4,)
(3,)


In [ ]:
freqcounts.head()

,id,daily,weekly,biweekly,monthly
0,1,57,47,5,1
1,2,42,41,7,1
2,3,55,51,2,1
3,4,47,42,7,0
4,5,50,40,7,1


In [ ]:
freqcounts.shape

(10, 5)

In [ ]:

prop_post = stats.dirichlet.rvs(size=1000, alpha=alpha+freqcounts.iloc[i,1:4])  # 1000 samples from the posterior distribution


In [ ]:






# Cumulative probabilities
prop_post_cumul = np.zeros((1000, 3))
prop_post_cumul[:, 0] = prop_post[:, 0]  # first column is Daily group only
prop_post_cumul[:, 1] = prop_post[:, 0] + prop_post[:, 1]  # second column is Daily group + Weekly group
prop_post_cumul[:, 2] = prop_post[:, 0] + prop_post[:, 1] + prop_post[:, 2]  # Third column is Daily + Weekly + Biweekly

np.apply_along_axis(lambda x: np.quantile(x, q=[0.025, 0.975]), axis=0, arr=prop_post)  # credible sets for each individual group
np.apply_along_axis(lambda x: np.quantile(x, q=[0.025, 0.975]), axis=0, arr=prop_post_cumul)  # credible sets for cumulative probabilities
# I suggest looking at the lower end of the credible set from the cumulative probabilities
# Just make sure to order the columns in increasing order from least time (day) to most time (month)

# Histograms of cumulative probabilities
plt.figure(figsize=(10, 8))
plt.subplot(2, 2, 1)
plt.hist(prop_post_cumul[:, 0], bins=30, density=True)
plt.xlabel('p1')
plt.title('Daily')
plt.axvline(x=np.mean(prop_post_cumul[:, 0]), color='r', linestyle='--')
plt.subplot(2, 2, 2)
plt.hist(prop_post_cumul[:, 1], bins=30, density=True)
plt.xlabel('p1')
plt.title('Daily+Weekly')
plt.axvline(x=np.mean(prop_post_cumul[:, 1]), color='r', linestyle='--')
plt.subplot(2, 2, 3)
plt.hist(prop_post_cumul[:, 2], bins=30, density=True)
plt.xlabel('p1')
plt.title('Daily+Weekly+Biweekly')
plt.axvline(x=np.mean(prop_post_cumul[:, 2]), color='r', linestyle='--')
plt.tight_layout()
plt.show()


InvalidIndexError: ignored